Libaries Install:

`pip install matplotlib==3.3.2`

`pip install scipy==1.4.1`


In [78]:
# import the libraries
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [79]:
print("Running Path, use this to set your file path")
print(os.getcwd())

Running Path, use this to set your file path
c:\Users\kunal\Documents\AI-21\ai-21\PROJECTS\voice_recognition


In [80]:
local_file_path = './audioFiles/3.wav'

# Read the wav file (mono)
samplingFrequency, signalData = wavfile.read(local_file_path)
print("Sampling Frequency: {}".format(samplingFrequency))
print("Signal Data: {}".format(signalData))
print("lmao kunal trash")

Sampling Frequency: 8000
Signal Data: [   0    0    0 ...  -21 -222  -35]
lmao kunal trash


Now that we see that the signal data is in a ___ format. We can see how we can manipulate it to measure the variance.

Since we can't print it, lets see how long it is

In [81]:
print(len(signalData))
# What does this mean?

128448


In [82]:
print(len(signalData)/samplingFrequency)

16.056


In [83]:
positionValue = 29700
print("Signal Data for {} : {}".format(positionValue, signalData[positionValue]))

Signal Data for 29700 : 7


# Creating a threshold

In [84]:
# Threshold for all values that are between 50 and -50
sumNon = 0
ct = 0
for i in signalData:
    if not -50 < i < 50 : 
        sumNon+=abs(i)
        ct+=1
print(sumNon/ct)

1139.2267973240366


In [85]:
# Average of all values
sum(abs(signalData))/len(signalData)

667.0591834828101

In [86]:
totalSumNonZero = 0
ct = 0
for num in signalData:
    if num != 0:
        totalSumNonZero+=num
        ct+=1
averageNonZero = totalSumNonZero / ct
print("Average Non Zero: {}".format(averageNonZero))

Average Non Zero: 0.036234905400621485


In [87]:
totalSumNonZero = 0
ct = 0
for num in signalData:
    if num > 10:
        totalSumNonZero+=num
        ct+=1
averageNonZero = totalSumNonZero / ct
print("Average Non Zero: {}".format(averageNonZero))

Average Non Zero: 777.5411818099301


# Apply the threshold

In [88]:
signalDataNormalized = []
for i in signalData:
    if abs(i) > 2000:
        signalDataNormalized.append(i)
    else:
        signalDataNormalized.append(0) 

In [89]:
# WHY WILL THIS NOT WORK
positionsSpeak = []
runningStartPositionCt = 0
startedSpeaking = False
for pos in range(len(signalDataNormalized)):
    if abs(signalDataNormalized[pos]) > 1999 and startedSpeaking == False:
        runningStartPositionCt = pos
        startedSpeaking = True
        print("Started Speaking at {}".format(runningStartPositionCt))
    if abs(signalDataNormalized[pos]) == 0 and startedSpeaking == True:
        startedSpeaking = False
        print("Stopped Speaking at {}".format(pos))
        positionsSpeak.append([runningStartPositionCt, pos])
print(positionsSpeak)

Started Speaking at 4094
Stopped Speaking at 4104
Started Speaking at 4107
Stopped Speaking at 4125
Started Speaking at 4128
Stopped Speaking at 4138
Started Speaking at 4139
Stopped Speaking at 4140
Started Speaking at 4141
Stopped Speaking at 4148
Started Speaking at 4149
Stopped Speaking at 4150
Started Speaking at 4152
Stopped Speaking at 4164
Started Speaking at 4165
Stopped Speaking at 4173
Started Speaking at 4175
Stopped Speaking at 4199
Started Speaking at 4200
Stopped Speaking at 4225
Started Speaking at 4226
Stopped Speaking at 4242
Started Speaking at 4243
Stopped Speaking at 4276
Started Speaking at 4277
Stopped Speaking at 4293
Started Speaking at 4294
Stopped Speaking at 4319
Started Speaking at 4320
Stopped Speaking at 4321
Started Speaking at 4322
Stopped Speaking at 4327
Started Speaking at 4328
Stopped Speaking at 4335
Started Speaking at 4336
Stopped Speaking at 4378
Started Speaking at 4379
Stopped Speaking at 4393
Started Speaking at 4394
Stopped Speaking at 4402


In [90]:
positionsSpeak = []
runningStartPositionCt = 0
startedSpeaking = False
for pos in range(len(signalDataNormalized)):
    if abs(signalDataNormalized[pos]) > 1999 and startedSpeaking == False:
        runningStartPositionCt = pos
        startedSpeaking = True
        #print("Started Speaking at {}".format(runningStartPositionCt))
    if (abs(signalDataNormalized[pos]) == 0 and startedSpeaking == True):
        nextcouple = [a for a in signalDataNormalized[pos:pos+1000]]
        if all(v == 0 for v in nextcouple):
            startedSpeaking = False
            #print("Stopped Speaking at {}".format(pos))
            positionsSpeak.append([runningStartPositionCt, pos])
print(positionsSpeak)
print(len(positionsSpeak))

[[4094, 5446], [11390, 12440], [19011, 20246], [26946, 27859], [33649, 36483], [43794, 45125], [53312, 54470], [61911, 62299], [69924, 71330], [78453, 79684], [101762, 102392], [108613, 110160], [112684, 112805], [115263, 126454]]
14


In [93]:
600/8000

0.075

In [94]:
posSpeakBuffer = []
buffer = 600
for i in positionsSpeak:
    posSpeakBuffer.append([i[0]-buffer, i[1]+buffer])
print(posSpeakBuffer)

[[3494, 6046], [10790, 13040], [18411, 20846], [26346, 28459], [33049, 37083], [43194, 45725], [52712, 55070], [61311, 62899], [69324, 71930], [77853, 80284], [101162, 102992], [108013, 110760], [112084, 113405], [114663, 127054]]


In [ ]:
[[3494, 6046], [10790, 13040], [18411, 20846], [26346, 28459], [33049, 37083], [43194, 45725], [52712, 55070], [61311, 62899], [69324, 71930], [77853, 80284], [101162, 102992], [108013, 110760], [112084, 113405], [114663, 127054]]

# Trim File

In [71]:
from scipy.io import wavfile
import string

In [66]:
def trim_wav_seconds(originalWavPath, newWavPath , start, end):
    sampleRate, waveData = wavfile.read( originalWavPath )
    startSample = int(start * sampleRate)
    endSample = int(end * sampleRate)
    wavfile.write( newWavPath, sampleRate, waveData[startSample:endSample])
def trim_wav(originalWavPath, newWavPath , start, end):
    sampleRate, waveData = wavfile.read( originalWavPath )
    return wavfile.write(newWavPath, sampleRate, waveData[start:end])

In [65]:
trim_wav(local_file_path, local_file_path.replace(".wav", "_trimmed.wav"), 4094, 5446)

In [95]:
ct = 0
alphabet_list = list(string.ascii_lowercase.upper())
for i in posSpeakBuffer:
    newName = (local_file_path.replace(".wav", "") + "_" + alphabet_list[ct] + 
              "_(" + str(i[0]) + "-" + str(i[1]) + ").wav")
    trim_wav(local_file_path, newName, i[0], i[1])
    ct+=1

In [ ]:
"3.wav"
"3"
"3_"
"3_A"
"3_A_(3000-5000).wav"